# Heart Attack Prediction


### Import and Load Data

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns

Load dataset diunduh dari `https://www.kaggle.com/datasets/thxogg/heart-attack-classification-training-dataset/data`

In [5]:
dataset_path = "datasets/heart_attack_dataset.csv"
heart_attack_dataset = pd.read_csv(dataset_path)
heart_attack_dataset

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin,class
0,62,1,66,160,83,160.0,1.80,0.012,negative
1,21,1,96,98,46,296.0,6.75,1.060,positive
2,55,1,64,160,77,270.0,1.99,0.003,negative
3,64,1,70,120,55,270.0,13.87,0.122,positive
4,55,1,64,112,65,300.0,1.08,0.003,negative
...,...,...,...,...,...,...,...,...,...
1314,44,1,94,122,67,204.0,1.63,0.006,negative
1315,66,1,84,125,55,149.0,1.33,0.172,positive
1316,45,1,85,168,104,96.0,1.24,4.250,positive
1317,54,1,58,117,68,443.0,5.80,0.359,positive
